In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
data = pd.read_csv("sql_injection_dataset.csv")

In [4]:
X = data['input']
y = data['label']

In [5]:
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 most important features
X = vectorizer.fit_transform(X).toarray()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Neural Network

In [7]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),  # Dropout to prevent overfitting
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(
    X_train, y_train,
    epochs=1000,  # Number of iterations over the dataset
    batch_size=32,  # Number of samples per gradient update
    validation_data=(X_test, y_test)
)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7003 - val_accuracy: 0.2500 - val_loss: 0.7108
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3125 - loss: 0.6976 - val_accuracy: 0.2500 - val_loss: 0.7046
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5000 - loss: 0.6911 - val_accuracy: 0.5000 - val_loss: 0.6986
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.7500 - loss: 0.6729 - val_accuracy: 0.5000 - val_loss: 0.6925
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.7500 - loss: 0.6629 - val_accuracy: 0.5000 - val_loss: 0.6865
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.8125 - loss: 0.6675 - val_accuracy: 0.5000 - val_loss: 0.6809
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.8750 - loss: 0.6645 - val_accuracy: 0.5000 - val_loss: 0.6759
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 1.0000 - loss: 0.6460 - val_accuracy: 0.5000

KeyboardInterrupt: 

Evaluation

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.1955
Test Accuracy: 100.00%


Prediction

In [11]:
new_input = ["' OR 1=1--"]  # Malicious query
new_input_vectorized = vectorizer.transform(new_input).toarray()

In [12]:
prediction = model.predict(new_input_vectorized)
if prediction[0] > 0.5:
    print("Malicious query detected!")
else:
    print("Query is safe.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Malicious query detected!


Flask Block

In [13]:
from flask import Flask, request, jsonify
import logging
import pickle  # Assuming your AI model is saved as a pickle file

app = Flask(__name__)

In [14]:
blocklist = set()

In [15]:
def is_malicious_query(query):
    prediction = ai_model.predict([query])
    return prediction[0] == "malicious"

In [16]:
def is_malicious_query(query):
    # Pass the query to your AI model and get prediction
    prediction = ai_model.predict([query])  # Assuming your model supports `predict()`
    return prediction[0] == "malicious"  # Adjust based on your model's output

@app.before_request
def check_blocklist():
    client_ip = request.remote_addr
    if client_ip in blocklist:
        return jsonify({"error": "Your IP is blocked due to malicious activity."}), 403

@app.route
def analyze_query():
    client_ip = request.remote_addr
    data = request.json

    if 'query' not in data:
        return jsonify({"error": "Missing 'query' parameter."}), 400

    query = data['query']

    # Use AI model to check if query is malicious
    if is_malicious_query(query):
        # Add the IP to the blocklist
        blocklist.add(client_ip)
        logging.warning(f"Blocked IP {client_ip} for malicious activity.")
        return jsonify({"error": "Malicious query detected. Your IP is now blocked."}), 403

    return jsonify({"message": "Query is safe."}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
